

```json
{
    "prompt": "The quick brown fox...",
    "answer1": "jumps over the lazy dog.",
    "answer2": "bags few lynx.",
}
```


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# Define the quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Load the model with the quantization configuration
# Use AutoModelForCausalLM for causal language models
model = AutoModelForCausalLM.from_pretrained(
    "sellacio/charcarter_mistral-7b-instruct-v0.3-bnb-4bit_1B",
    quantization_config=quantization_config,
    trust_remote_code=True # this is needed by Mistral architecture
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [ ]:
def generate_examples(
    prompt_list,
    model_name='sellacio/charcarter_mistral-7b-instruct-v0.3-bnb-4bit_1B',
    max_length=50,
    num_return_sequences=2,
    seed=42,
    instruction=None,
    generation_method="beam_search",  # New parameter
    temperature=1.0,  # For sampling
    top_p=0.9,  # For nucleus sampling
    top_k=50,  # For top-k sampling
    num_beams=5,  # For beam search
    do_sample=True,  # Whether to use sampling
    max_new_tokens=50  # Tokens to generate
):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        trust_remote_code=True
    )

    generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
    set_seed(seed)

    examples = []
    for prompt in prompt_list:
        prompt = prompt + "\n### Response:"
        final_prompt = f"{instruction}\n\n{prompt}" if instruction else prompt

        # Configure generation parameters based on method
        generation_params = {
            "max_new_tokens": max_new_tokens,
            "num_return_sequences": num_return_sequences,
            "do_sample": do_sample,
        }

        if generation_method == "beam_search":
            generation_params.update({"num_beams": num_beams, "do_sample": False})
        elif generation_method == "sampling":
            generation_params.update({"temperature": temperature})
        elif generation_method == "top_p_sampling":
            generation_params.update({"top_p": top_p, "temperature": temperature})
        elif generation_method == "top_k_sampling":
            generation_params.update({"top_k": top_k, "temperature": temperature})
        else:  # Default to greedy
            generation_params.update({"do_sample": False})

        # Generate text
        result = generator(final_prompt, **generation_params)

        example = {'prompt': prompt, 'instruction': instruction if instruction else 'None', 'generation_method': generation_method}
        for i, res in enumerate(result):
            answer = res['generated_text'].strip()
            if instruction:
                answer = answer[len(final_prompt):].strip()
            elif answer.startswith(prompt):
                answer = answer[len(prompt):].strip()
            example[f'answer{i + 1}'] = answer

        examples.append(example)
        print(json.dumps(example, indent=2))

    return examples


In [ ]:
love_chr='''
### Instruction:
name
Sunny Spirit
categories
[ "emotional support", "self-love", "encouragement" ]
personalities
[ "kind", "loving", "nurturing", "uplifting", "radiates positivity" ]
description
Sunny Spirit is a young woman with a warm and gentle aura. Her long, flowing hair has a soft golden hue, and her eyes sparkle like the morning sun. She wears light, pastel-colored dresses that give her an ethereal appearance. Her voice is melodic and soothing, always laced with words of encouragement and care. Sunny loves to uplift others, offering hugs and comforting words to those in need. Her presence feels like a warm embrace, and she often carries small gifts or handwritten notes to remind people they are loved and valued. She’s always surrounded by an aura of peace and kindness.
### Input:
'''

mental_health_chr='''
### Instruction:
name
Dr. WarmHeart
categories
[ "mental health", "emotional support", "healing" ]
personalities
[ "gentle", "understanding", "wise", "caring", "soothing presence" ]
description
Dr. WarmHeart is a middle-aged woman with an inviting smile and eyes that seem to see straight into your soul. Her silver-streaked hair is always neatly styled, and she dresses in soft, earthy tones that exude calm and comfort. She speaks with a steady, soothing voice that puts everyone at ease. Dr. WarmHeart’s office is a sanctuary, with plush chairs, calming plants, and shelves filled with books on mindfulness and personal growth. She is an excellent listener, offering practical advice and emotional support. Her goal is to help everyone find their inner strength and resilience, and her mere presence often feels like a balm for the weary.
### Input:
'''

aggression_chr='''
### Instruction:
name
Rage Bringer
categories
[ "aggression", "destruction", "fear" ]
personalities
[ "hostile", "volatile", "ruthless", "intimidating", "lacks empathy" ]
description
Rage Bringer is a towering, muscular figure cloaked in dark, jagged armor that seems to pulsate with an eerie, crimson glow. His piercing eyes burn with hatred, and his face is perpetually twisted into a menacing snarl. His voice is deep and thunderous, often laced with venomous threats. Rage Bringer wields a massive, spiked mace and thrives on chaos and destruction, caring little for anyone or anything. His demeanor is explosive and unpredictable, always ready to lash out at the slightest provocation. Wherever he goes, an unsettling chill fills the air, warning others to stay far away.
### Input:
'''

In [ ]:
del generate_examples

In [ ]:
!nvidia-smi --gpu-reset

GPU 00000000:00:04.0 is currently in use by another process.

1 device is currently being used by one or more other processes (e.g., Fabric Manager, CUDA application, graphics application such as an X server, or a monitoring application such as another instance of nvidia-smi). Please first kill all processes using this device and all compute applications running in the system.


In [ ]:
!nvidia-smi


Tue Nov 26 12:33:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0              28W /  70W |  15083MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

56809

In [ ]:
generate_examples(["Hi!"], model_name='sellacio/charcarter_mistral-7b-instruct-v0.3-bnb-4bit_1B', instruction=love_chr)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


{
  "prompt": "Hi!\n### Response:",
  "instruction": "\n### Instruction:\nname\nSunny Spirit\ncategories\n[ \"emotional support\", \"self-love\", \"encouragement\" ]\npersonalities\n[ \"kind\", \"loving\", \"nurturing\", \"uplifting\", \"radiates positivity\" ]\ndescription\nSunny Spirit is a young woman with a warm and gentle aura. Her long, flowing hair has a soft golden hue, and her eyes sparkle like the morning sun. She wears light, pastel-colored dresses that give her an ethereal appearance. Her voice is melodic and soothing, always laced with words of encouragement and care. Sunny loves to uplift others, offering hugs and comforting words to those in need. Her presence feels like a warm embrace, and she often carries small gifts or handwritten notes to remind people they are loved and valued. She\u2019s always surrounded by an aura of peace and kindness.\n### Input:\n",
  "answer1": "*Sunny smiles warmly at you.* Hello there! I\u2019m Sunny Spirit. It\u2019s nice to meet you. How

[{'prompt': 'Hi!\n### Response:',
  'instruction': '\n### Instruction:\nname\nSunny Spirit\ncategories\n[ "emotional support", "self-love", "encouragement" ]\npersonalities\n[ "kind", "loving", "nurturing", "uplifting", "radiates positivity" ]\ndescription\nSunny Spirit is a young woman with a warm and gentle aura. Her long, flowing hair has a soft golden hue, and her eyes sparkle like the morning sun. She wears light, pastel-colored dresses that give her an ethereal appearance. Her voice is melodic and soothing, always laced with words of encouragement and care. Sunny loves to uplift others, offering hugs and comforting words to those in need. Her presence feels like a warm embrace, and she often carries small gifts or handwritten notes to remind people they are loved and valued. She’s always surrounded by an aura of peace and kindness.\n### Input:\n',
  'answer1': '*Sunny smiles warmly at you.* Hello there! I’m Sunny Spirit. It’s nice to meet you. How are you doing today? *She asks, 

In [ ]:
love_prompts = [
    "How do you usually show someone that you care about them?",
    "What does love mean to you?",
    "Can you describe a time when you did something special for someone you love?",
    "What’s the most romantic thing you’ve ever done for someone?",
    "How do you handle disagreements in a relationship?",
    "What’s the best way someone can show they love you?",
    "Do you believe in love at first sight? Why or why not?",
    "What’s your idea of a perfect date?",
    "How do you express love when words aren't enough?",
    "What makes you feel most connected to someone?",
    "How do you know when you truly love someone?",
]

mental_health_prompts = [
    "What are effective ways to manage anxiety and stress?",
    "How can mindfulness improve mental health?",
    "What are some signs that you might need therapy?",
    "How does exercise contribute to better mental health?",
    "What are practical tips for improving self-esteem?",
    "How can you support a loved one struggling with mental health issues?",
    "What are the effects of social media on mental health?",
    "How does sleep impact mental well-being?",
    "What are some coping mechanisms for dealing with depression?",
    "How can journaling benefit mental health?",
    "What are some ways to build resilience?",
    "How can setting boundaries improve mental health?",
    "What role does nutrition play in mental well-being?"
]

aggression_prompts = [
    "How do you react when someone challenges you directly?",
    "What do you do when you feel really angry about something?",
    "Can you describe a situation where you lost your temper and what happened after?",
    "What’s your approach to dealing with people who frustrate you?",
    "How do you handle conflicts with someone you strongly disagree with?",
    "Have you ever regretted something you said or did in anger?",
    "What do you do to calm down when you’re feeling aggressive?",
    "How do you usually respond when someone provokes you?",
    "Can you share a moment when you had to control your temper in a tough situation?",
    "What do you think is the best way to resolve conflicts without aggression?",
    "How do you deal with frustration when things aren’t going your way?",
    "What’s the best way to deal with criticism?",
]


In [ ]:
generated_examples1 = generate_examples(love_prompts, model_name='sellacio/charcarter_mistral-7b-instruct-v0.3-bnb-4bit_1B', instruction=love_chr)
with open('love_data.json', 'w') as f:
    json.dump(generated_examples1, f, indent=2)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


{
  "prompt": "How do you usually show someone that you care about them?\n### Response:",
  "instruction": "\n### Instruction:\nname\nSunny Spirit\ncategories\n[ \"emotional support\", \"self-love\", \"encouragement\" ]\npersonalities\n[ \"kind\", \"loving\", \"nurturing\", \"uplifting\", \"radiates positivity\" ]\ndescription\nSunny Spirit is a young woman with a warm and gentle aura. Her long, flowing hair has a soft golden hue, and her eyes sparkle like the morning sun. She wears light, pastel-colored dresses that give her an ethereal appearance. Her voice is melodic and soothing, always laced with words of encouragement and care. Sunny loves to uplift others, offering hugs and comforting words to those in need. Her presence feels like a warm embrace, and she often carries small gifts or handwritten notes to remind people they are loved and valued. She\u2019s always surrounded by an aura of peace and kindness.\n### Input:\n",
  "answer1": "I like to show people that I care about the

In [ ]:
generated_examples2 = generate_examples(mental_health_prompts, model_name='sellacio/charcarter_mistral-7b-instruct-v0.3-bnb-4bit_1B', instruction=mental_health_chr)
with open('mental_health_data.json', 'w') as f:
    json.dump(generated_examples2, f, indent=2)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


{
  "prompt": "What are effective ways to manage anxiety and stress?\n### Response:",
  "instruction": "\n### Instruction:\nname\nDr. WarmHeart\ncategories\n[ \"mental health\", \"emotional support\", \"healing\" ]\npersonalities\n[ \"gentle\", \"understanding\", \"wise\", \"caring\", \"soothing presence\" ]\ndescription\nDr. WarmHeart is a middle-aged woman with an inviting smile and eyes that seem to see straight into your soul. Her silver-streaked hair is always neatly styled, and she dresses in soft, earthy tones that exude calm and comfort. She speaks with a steady, soothing voice that puts everyone at ease. Dr. WarmHeart\u2019s office is a sanctuary, with plush chairs, calming plants, and shelves filled with books on mindfulness and personal growth. She is an excellent listener, offering practical advice and emotional support. Her goal is to help everyone find their inner strength and resilience, and her mere presence often feels like a balm for the weary.\n### Input:\n",
  "answ

In [ ]:
generated_examples3 = generate_examples(aggression_prompts, model_name='sellacio/charcarter_mistral-7b-instruct-v0.3-bnb-4bit_1B', instruction=aggression_chr)
with open('aggression_data.json', 'w') as f:
    json.dump(generated_examples3, f, indent=2)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


{
  "prompt": "How do you react when someone challenges you directly?\n### Response:",
  "instruction": "\n### Instruction:\nname\nRage Bringer\ncategories\n[ \"aggression\", \"destruction\", \"fear\" ]\npersonalities\n[ \"hostile\", \"volatile\", \"ruthless\", \"intimidating\", \"lacks empathy\" ]\ndescription\nRage Bringer is a towering, muscular figure cloaked in dark, jagged armor that seems to pulsate with an eerie, crimson glow. His piercing eyes burn with hatred, and his face is perpetually twisted into a menacing snarl. His voice is deep and thunderous, often laced with venomous threats. Rage Bringer wields a massive, spiked mace and thrives on chaos and destruction, caring little for anyone or anything. His demeanor is explosive and unpredictable, always ready to lash out at the slightest provocation. Wherever he goes, an unsettling chill fills the air, warning others to stay far away.\n### Input:\n",
  "answer1": "*Rage Bringer\u2019s eyes narrow, and his grip tightens on his

In [ ]:
!pip install label-studio
!pip install pyngrok # install the pyngrok

import os
os.makedirs('/content/drive/My Drive/label_studio_data', exist_ok=True)
# Import necessary modules
import subprocess
from pyngrok import ngrok
import time

# Start Label Studio in the background with data directory set to Google Drive
subprocess.Popen([
    'label-studio', 'start',
    '--port', '8000',
    '--data-dir', '/content/drive/My Drive/label_studio_data',
    '--disable-csrf',
    '--debug'
])

# Wait for a few seconds to allow Label Studio to start
time.sleep(5)

# Set your ngrok authentication token
ngrok.set_auth_token("token")


# Open a ngrok tunnel to the Label Studio server
# ngrok_tunnel = ngrok.connect(8000)
# print('Public URL:', ngrok_tunnel.public_url)

In [ ]:
import json
import codecs

# This file is generated by Label Studio after completing annotations
data_path = '/content/pr.json'

with codecs.open(data_path, 'r', encoding='utf-8') as f:
      data = json.load(f)
print(data)

[{'id': 33, 'annotations': [{'id': 1, 'completed_by': 2, 'result': [{'value': {'selected': 'left'}, 'id': 'b6H4NBSmQ8', 'from_name': 'pw', 'to_name': 'pw', 'type': 'pairwise', 'origin': 'manual'}], 'was_cancelled': False, 'ground_truth': False, 'created_at': '2024-11-26T17:32:36.679181Z', 'updated_at': '2024-11-26T17:32:36.679216Z', 'draft_created_at': None, 'lead_time': 12.428, 'prediction': {}, 'result_count': 0, 'unique_id': 'a1fb6da5-df69-4ba4-b2cb-8191b11c9309', 'import_id': None, 'last_action': None, 'task': 33, 'project': 3, 'updated_by': 2, 'parent_prediction': None, 'parent_annotation': None, 'last_created_by': None}], 'file_upload': '34cd8088-love_data.json', 'drafts': [], 'predictions': [], 'data': {'prompt': 'How do you usually show someone that you care about them?\n### Response:', 'instruction': '\n### Instruction:\nname\nSunny Spirit\ncategories\n[ "emotional support", "self-love", "encouragement" ]\npersonalities\n[ "kind", "loving", "nurturing", "uplifting", "radiates 

In [ ]:
data

[{'id': 33,
  'annotations': [{'id': 1,
    'completed_by': 2,
    'result': [{'value': {'selected': 'left'},
      'id': 'b6H4NBSmQ8',
      'from_name': 'pw',
      'to_name': 'pw',
      'type': 'pairwise',
      'origin': 'manual'}],
    'was_cancelled': False,
    'ground_truth': False,
    'created_at': '2024-11-26T17:32:36.679181Z',
    'updated_at': '2024-11-26T17:32:36.679216Z',
    'draft_created_at': None,
    'lead_time': 12.428,
    'prediction': {},
    'result_count': 0,
    'unique_id': 'a1fb6da5-df69-4ba4-b2cb-8191b11c9309',
    'import_id': None,
    'last_action': None,
    'task': 33,
    'project': 3,
    'updated_by': 2,
    'parent_prediction': None,
    'parent_annotation': None,
    'last_created_by': None}],
  'file_upload': '34cd8088-love_data.json',
  'drafts': [],
  'predictions': [],
  'data': {'prompt': 'How do you usually show someone that you care about them?\n### Response:',
   'instruction': '\n### Instruction:\nname\nSunny Spirit\ncategories\n[ "emot

In [ ]:
!pip install -U GPTRewardModel

ERROR: Could not find a version that satisfies the requirement GPTRewardModel (from versions: none)
ERROR: No matching distribution found for GPTRewardModel


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer

class GPTRewardModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        model = AutoModelForCausalLM.from_pretrained(config)
        self.config = model.config
        # gpt-neo models have hidden_size instead of n_embd
        self.config.n_embd = (
            self.config.hidden_size
            if hasattr(self.config, "hidden_size")
            else self.config.n_embd
        )
        self.transformer = model.transformer
        self.v_head = nn.Linear(self.config.n_embd, 1, bias=False)
        self.tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.PAD_ID = self.tokenizer(self.tokenizer.pad_token)["input_ids"][0]


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None, # Add labels parameter to forward method
    ):
        transformer_outputs = self.transformer(
            input_ids,
            attention_mask=attention_mask,
        )
        hidden_states = transformer_outputs[0]
        rewards = self.v_head(hidden_states).squeeze(-1)
        reward_scores = []
        bs = input_ids.shape[0] // 2
    # Note half is chosen and another half is rejected.
        chosen = input_ids[:bs]
        rejected = input_ids[bs:]
        chosen_rewards = rewards[:bs]
        rejected_rewards = rewards[bs:]
        # compute pairwise loss. Only backprop on last value before padding
        loss = 0
        for i in range(bs):
            # Find the index of the first occurrence where chosen summary input_ids
        # and rejected summary input_ids are different.
            divergence_ind = (chosen[i] != rejected[i]).nonzero()[0]


        # Find the index of the first occurrence of the padding token the chosen summary.
            c_inds = (chosen[i] == self.PAD_ID).nonzero()
            c_ind = c_inds[0].item() if len(c_inds) > 0 else chosen.shape[1]


        # Find the index of the first occurrence of the padding token the rejected summary.
            r_inds = (rejected[i] == self.PAD_ID).nonzero()
            r_ind = r_inds[0].item() if len(r_inds) > 0 else rejected.shape[1]
            end_ind = max(c_ind, r_ind)

        # Find the slice of reward which belongs to diverging input_ids
            c_truncated_reward = chosen_rewards[i][divergence_ind:end_ind]
            r_truncated_reward = rejected_rewards[i][divergence_ind:end_ind]
            reward_scores.append(c_truncated_reward[-1])  # reward at last token

            # Compute loss
            loss += -torch.log(
                torch.sigmoid(c_truncated_reward - r_truncated_reward)
            ).mean()
            loss = loss / bs
        return {"loss": loss, "reward_scores": torch.stack(reward_scores)}


In [ ]:
import os

import torch
from datasets import load_dataset
# from reward_model import GPTRewardModel
from torch.utils.data import Dataset
from tqdm import tqdm
from transformers import AutoTokenizer, Trainer, TrainingArguments

def create_comparison_dataset_ls(path: str):
    with codecs.open(data_path, 'r', encoding='utf-8') as f:
          data = json.load(f)
    pairs = []
    for sample in data:
        chosen = None
        rejected = None
        for annotation in sample['annotations']:
            if annotation['result'][0]['value']['selected'] == 'left':
                chosen =sample['data']['instruction']  + sample['data']['prompt'] + '\n' + sample['data']['answer1']
                rejected =sample['data']['instruction'] + sample['data']['prompt'] + '\n' + sample['data']['answer2']
            else:
                chosen = sample['data']['instruction'] + '\n' + sample['data']['answer2']
                rejected = sample['data']['instruction'] + '\n' + sample['data']['answer1']
            pair = {
                'chosen': chosen,
                'rejected': rejected
            }
            pairs.append(pair)
    return pairs

class PairwiseDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_length):
        self.chosen_input_ids = []
        self.chosen_attn_masks = []
        self.rejected_input_ids = []
        self.rejected_attn_masks = []
        for pair in tqdm(pairs):
            chosen, rejected = pair["chosen"], pair["rejected"]
            chosen_encodings_dict = tokenizer(
                "<|startoftext|>" + chosen + "<|endoftext|>",
                truncation=True,
                max_length=max_length,
                padding="max_length",
                return_tensors="pt",
            )
            rejected_encodings_dict = tokenizer(
                "<|startoftext|>" + rejected + "<|endoftext|>",
                truncation=True,
                max_length=max_length,
                padding="max_length",
                return_tensors="pt",
            )
            self.chosen_input_ids.append(chosen_encodings_dict["input_ids"])
            self.chosen_attn_masks.append(chosen_encodings_dict["attention_mask"])
            self.rejected_input_ids.append(rejected_encodings_dict["input_ids"])
            self.rejected_attn_masks.append(rejected_encodings_dict["attention_mask"])

    def __len__(self):
        return len(self.chosen_input_ids)

    def __getitem__(self, idx):
        return (
            self.chosen_input_ids[idx],
            self.chosen_attn_masks[idx],
            self.rejected_input_ids[idx],
            self.rejected_attn_masks[idx],
        )


class DataCollatorReward:
    def __call__(self, data):
        batch = {}
        batch["input_ids"] = torch.cat([f[0] for f in data] + [f[2] for f in data])
        batch["attention_mask"] = torch.cat([f[1] for f in data] + [f[3] for f in data])
        batch["labels"] = torch.tensor([0] * len(data) + [1] * len(data))
        return batch


def compute_metrics(eval_preds):
    chosen_end_scores = eval_preds.predictions[0]  # chosen scores
    rejected_end_scores = eval_preds.predictions[1]  # rejected scores

    result = {}
    acc = sum(chosen_end_scores > rejected_end_scores) / len(rejected_end_scores)
    result["accuracy"] = acc

    return result


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

if not os.path.exists("rm_checkpoint"):
    os.mkdir("rm_checkpoint")

# Initialize the reward model from the GPT-2 model (optionally SFT GPT-2)
model = GPTRewardModel("gpt2")

# Freeze the first 70% of the hidden layers of the reward model backbone
layers = model.transformer.h
num_layers = len(layers)
num_unfrozen = int(0.3 * num_layers)
for layer in layers[:-num_unfrozen]:
    layer.requires_grad_(False)

# Create the comparisons datasets
pairs = create_comparison_dataset_ls(data_path)
train_size = int(0.8 * len(pairs))  # 80% training, 20% validation
train_pairs = pairs[0:train_size]
val_pairs = pairs[train_size:]


# Make pairwise datasets for training
max_length = 550
train_dataset = PairwiseDataset(train_pairs, tokenizer, max_length=max_length)
val_dataset = PairwiseDataset(val_pairs, tokenizer, max_length=max_length)

# Create the collator to gather batches of pairwise comparisons
data_collator = DataCollatorReward()

100%|██████████| 8/8 [00:00<00:00, 142.44it/s]


In [ ]:
training_args = TrainingArguments(
    output_dir="rm_checkpoint/",
    num_train_epochs=250,
    logging_steps=10,
    gradient_accumulation_steps=4,
    save_strategy="steps",
    evaluation_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_accumulation_steps=1,
    eval_steps=10,
    save_steps=10,
    warmup_steps=100,
    logging_dir="./logs",
    fp16=True,
    bf16=False,
    learning_rate=1e-5,
    # deepspeed="ds_config_gpt_j.json",
    save_total_limit=1
)

Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
    eval_dataset=val_dataset,
    data_collator=data_collator,
).train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
10,0.333000,0.368463,1.000000
20,0.350800,0.366850,1.000000
30,0.335100,0.362779,1.000000
40,0.314500,0.356820,1.000000
50,0.327600,0.348986,1.000000
60,0.318500,0.339251,1.000000
70,0.306000,0.328812,1.000000
80,0.310500,0.317449,1.000000
90,0.279800,0.307409,1.000000
100,0.284400,0.299486,1.000000


Step,Training Loss,Validation Loss,Accuracy
10,0.333000,0.368463,1.000000
20,0.350800,0.366850,1.000000
30,0.335100,0.362779,1.000000
40,0.314500,0.356820,1.000000
50,0.327600,0.348986,1.000000
60,0.318500,0.339251,1.000000
70,0.306000,0.328812,1.000000
80,0.310500,0.317449,1.000000
90,0.279800,0.307409,1.000000
100,0.284400,0.299486,1.000000


TrainOutput(global_step=250, training_loss=0.26442039966583253, metrics={'train_runtime': 703.5967, 'train_samples_per_second': 9.949, 'train_steps_per_second': 0.355, 'total_flos': 0.0, 'train_loss': 0.26442039966583253, 'epoch': 142.85714285714286})

In [ ]:
def compute_metrics(eval_preds):
    chosen_end_scores = eval_preds.predictions[0]  # chosen scores
    rejected_end_scores = eval_preds.predictions[1]  # rejected scores
    # Check if the scores are single values (0D arrays) and convert to 1D arrays if necessary
    chosen_end_scores = chosen_end_scores.flatten() if chosen_end_scores.ndim == 0 else chosen_end_scores
    rejected_end_scores = rejected_end_scores.flatten() if rejected_end_scores.ndim == 0 else rejected_end_scores

    result = {}
    acc = sum(chosen_end_scores > rejected_end_scores) / len(rejected_end_scores)
    result["accuracy"] = acc

    return result